## Install and Import

In [1]:
%pip install -q rdkit pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from rdkit import Chem, RDLogger
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
import numpy as np
import tensorflow as tf
from tensorflow import keras

RDLogger.DisableLog("rdApp.*")

## Dataset

In [3]:
class DataHelper():
    def __init__(self):
        csv_path = tf.keras.utils.get_file("qm9.csv", "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")
        self.data = list()
        with open(csv_path, "r")  as fin:
            for line in fin.readlines()[1:]:
                self.data.append(line.split(",")[1])

    def __getitem__(self, idx: int):
        smiles = self.data[idx]
        print(f"SMILES: {smiles}")
        mol = Chem.MolFromSmiles(smiles)
        print(f"Number of Heavy Atoms: {mol.GetNumHeavyAtoms()}")
        return mol

In [4]:
DH = DataHelper()

29856825/29856825 [==============================] - 2s 0us/step


## Utilities

In [6]:
atom_mapping = {
    "C": 0,
    0: "C",
    "N": 1,
    1: "N",
    "O": 2,
    2: "O",
    "F": 3,
    3: "F",
}

bond_mapping = {
    "SINGLE": 0,
    0: Chem.BondType.SINGLE,
    "DOUBLE": 1,
    1: Chem.BondType.DOUBLE,
    "TRIPLE": 2,
    2: Chem.BondType.TRIPLE,
    "AROMATIC": 3,
    3: Chem.BondType.AROMATIC,
}

NUM_ATOMS = 9  # Maximum number of atoms
ATOM_DIM = 4 + 1  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 64  # Size of the latent space


def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # loop over each atom in molecule
    for atom in molecule.GetAtoms():
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]
        # loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type_idx = bond_mapping[bond.GetBondType().name]
            adjacency[bond_type_idx, [i, j], [j, i]] = 1

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule

## Generate Training Set

In [7]:
adjacency_tensor, feature_tensor = [], []
for smiles in DH.data[::10]:
    adjacency, features = smiles_to_graph(smiles)
    adjacency_tensor.append(adjacency)
    feature_tensor.append(features)

adjacency_tensor = np.array(adjacency_tensor)
feature_tensor = np.array(feature_tensor)

print("adjacency_tensor.shape =", adjacency_tensor.shape)
print("feature_tensor.shape =", feature_tensor.shape)

adjacency_tensor.shape = (13389, 5, 9, 9)
feature_tensor.shape = (13389, 9, 5)


## New Generator

### Quantum Dense Layer

#### Extra Utilities/Imports

In [8]:
%pip install -q qiskit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [9]:
import qiskit

In [10]:
from qiskit import transpile, assemble, QuantumRegister, QuantumCircuit
from qiskit.providers.ibmq import least_busy
from qiskit.providers.ibmq.job import job_monitor
from qiskit.tools import backend_monitor

#### IDK what these functions do

#### Quantum Dense Layer

In [178]:
class QuantumLayer(keras.layers.Layer):
    def __init__(self, qubits=16, instructions=None, shots=2, use_parameter_shift_gradient_flow=False):
        super(QuantumLayer, self).__init__()
        self.use_parameter_shift_gradient_flow = use_parameter_shift_gradient_flow
        self.qubits = qubits
        self.instructions = instructions
        self.tensor_history = []
        self.shots = shots
        self.circuit = QiskitCircuitModule(self.qubits, instructions=self.instructions, shots=self.shots)

    def build(self, input_shape):
        kernel_p_initialisation = tf.random_normal_initializer()
        self.kernel_p = tf.Variable(name="kernel_p", initial_value=kernel_p_initialisation(shape=(input_shape[-1], self.qubits), dtype='float32'), trainable=True)
        kernel_phi_initialisation = tf.zeros_initializer()
        self.kernel_phi = tf.Variable(name="kernel_phi", initial_value=kernel_phi_initialisation(shape=(self.qubits,), dtype='float32'), trainable=False)

    def call(self, inputs):
        if not self.use_parameter_shift_gradient_flow:
            output = tf.matmul(inputs, self.kernel_p)
            print(output)
            qubit_output = self.circuit.quantum_execute(tf.reshape(output, [1, self.qubits]), self.kernel_phi)
            qubit_output = tf.reshape(tf.convert_to_tensor(qubit_output), (1, 1, self.qubits))
            output += (qubit_output - output)
        else: output = self.quantum_flow(inputs)
        return output

    @tf.custom_gradient
    def quantum_flow(self, x):
        output = tf.matmul(x, self.kernel_p)
        qubit_output = tf.reshape(tf.convert_to_tensor(self.circuit.quantum_execute(tf.reshape(output, [1, self.qubits]), self.kernel_phi)), (1, 1, self.qubits))
        output = qubit_output

        def grad(dy, variables=None):
            shift = np.pi / 2
            shift_right = x + np.ones(x.shape) * shift
            shift_left = x - np.ones(x.shape) * shift
            input_left = tf.matmul(shift_left, self.kernel_p)
            input_right = tf.matmul(shift_right, self.kernel_p)
            output_right = self.circuit.quantum_execute(tf.reshape(input_right, [1, self.qubits]), self.kernel_phi)
            output_left = self.circuit.quantum_execute(tf.reshape(input_left, [1, self.qubits]), self.kernel_phi)
            quantum_gradient = [output_right[i] - output_left[i] for i in range(len(output_right))]
            input_gradient = dy * quantum_gradient
            dy_input_gradient = tf.reshape(tf.matmul(input_gradient, tf.transpose(self.kernel_p)), shape=[1, 1, x.get_shape().as_list()[-1]])
            grd_w = []
            for i in range(self.qubits):
                w = self.kernel_p[:, i]
                w += dy_input_gradient
                grd_w.append(w)
            tf_grd_w = tf.convert_to_tensor(grd_w)
            tf_grd_w = tf.reshape(tf_grd_w, shape=(x.get_shape().as_list()[-1], self.qubits))
            return dy_input_gradient, [tf_grd_w]

        return output, grad

In [110]:
class QiskitCircuitModule:
    def __init__(self, qubits, instructions=None, execute_on_IBMQ=False, shots=2):
        self.qubit_num = qubits
        self.instructions = instructions
        if not self.instructions:
            self.instructions = self.null_circuit(self.qubit_num)
        self.probabilities = tf.constant([[0.5] * self.qubit_num])
        self.phase_probabilities = tf.constant([1] * self.qubit_num)
        self.layer = self.superposition_qubits(self.probabilities, self.phase_probabilities)
        self.layer.append(self.instructions, range(self.qubit_num))
        self.layer.measure_all()
        self.backend = qiskit.Aer.get_backend('aer_simulator')
        self.shots = shots

    def p_to_angle(self, p):
        angle = 2 * np.arccos(np.sqrt(p))
        return angle

    def superposition_qubits(self, probabilities: tf.Tensor, phases: tf.Tensor):
        layer = qiskit.QuantumCircuit(self.qubit_num)
        reshaped_probabilities = tf.reshape(probabilities, [self.qubit_num])
        reshaped_phases = tf.reshape(phases, [self.qubit_num])
        static_probabilities = tf.get_static_value(reshaped_probabilities[:])
        static_phases = tf.get_static_value(reshaped_phases[:])
        for i in range(len(static_probabilities)):
            p = static_probabilities[i]
        #for ix, p in enumerate(static_probabilities):
            p = np.abs(p)
            theta = self.p_to_angle(p)
            phi = self.p_to_angle(static_phases[i])
            layer.u(theta, phi, 0, i)
        return layer

    def quantum_execute(self, probabilities, phases):
        self.layer = self.superposition_qubits(probabilities, phases)
        self.layer.append(self.instructions, range(self.qubit_num))
        self.layer.measure_all()
        transpiled_circuit = transpile(self.layer, self.backend)
        quantum_job_object = assemble(transpiled_circuit, shots=self.shots)
        quantum_job = self.backend.run(quantum_job_object)
        result = quantum_job.result().get_counts()
        qubit_set_probabilities = self.calculate_qubit_set_probabilities(result)
        return qubit_set_probabilities

    def calculate_qubit_set_probabilities(self, quantum_job_result):
        qubit_set_probabilities = [0] * self.qubit_num
        for state_result, count in quantum_job_result.items():
            for ix, q in enumerate(state_result):
                if q == '1':
                    qubit_set_probabilities[ix] += count
        sum_counts = sum(qubit_set_probabilities)
        if not sum_counts == 0:
            qubit_set_probabilities = [i/sum_counts for i in qubit_set_probabilities]
        return qubit_set_probabilities

    def null_circuit(self, qubits):
        try:
            gate_register = QuantumRegister(qubits, 'q')
            gate_circuit = QuantumCircuit(gate_register, name='sub_circuit')
            gate_instructions = gate_circuit.to_instruction()
        except Exception as e: raise RuntimeError("🅵🆄🅲🅺!")
        return gate_instructions

#### Allah be praised IT WORKS

In [174]:
sus = tf.constant([[0.345], ])

In [199]:
class GraphGenerator(keras.Model):
    def __init__(self, dense_units, dropout_rate, latent_dim, adjacency_shape, feature_shape, param_shift=False,):
        super(GraphGenerator, self).__init__()
        self.z = keras.layers.Dense(latent_dim, input_shape=(1,2))
        self.ql = QuantumLayer(dense_units, use_parameter_shift_gradient_flow=param_shift)
        self.dropout = keras.layers.Dropout(dropout_rate)
        self.latent_dim = latent_dim
        self.adjacency_shape = adjacency_shape
        self.feature_shape = feature_shape

    def call(self, input_tensor):
        x = self.z(input_tensor)
        x = self.ql(x)
        x = keras.activations.tanh(x)
        x = self.dropout(x)
        x_adjacency = keras.layers.Dense(tf.math.reduce_prod(self.adjacency_shape))(x)
        x_adjacency = keras.layers.Reshape(self.adjacency_shape)(x_adjacency)
        x_adjacency = (x_adjacency + tf.transpose(x_adjacency, (0, 1, 3, 2))) / 2
        x_adjacency = keras.layers.Softmax(axis=1)(x_adjacency)
        x_features = keras.layers.Dense(tf.math.reduce_prod(self.feature_shape))(x)
        x_features = keras.layers.Reshape(self.feature_shape)(x_features)
        x_features = keras.layers.Softmax(axis=2)(x_features)
        return [x_adjacency, x_features]


In [200]:
AIYAH = GraphGenerator(16, 0.2, 64, (BOND_DIM, NUM_ATOMS, NUM_ATOMS), (NUM_ATOMS, ATOM_DIM), False)

In [201]:
holy_shit_it_works = AIYAH(sus)

tf.Tensor(
[[ 0.04611792 -0.01190395 -0.00582716  0.02072552  0.0495905  -0.00206493
  -0.02366902  0.01037156  0.00390775  0.00630414 -0.02273801 -0.00925849
   0.00098659  0.011702    0.01168314  0.04228592]], shape=(1, 16), dtype=float32)


<ipython-input-110-18d72db1aad0>:40: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  quantum_job = self.backend.run(quantum_job_object)


In [202]:
AIYAH.summary()

Model: "graph_generator_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            multiple                  128       
                                                                 
 quantum_layer_37 (QuantumLa  multiple                 1040      
 yer)                                                            
                                                                 
 dropout_34 (Dropout)        multiple                  0         
                                                                 
Total params: 1,168
Trainable params: 1,152
Non-trainable params: 16
_________________________________________________________________


## Graph Discriminator

### Relational Graph Convolution Layer

In [45]:
class RelationalGraphConvLayer(keras.layers.Layer):
    def __init__(
        self,
        units=128,
        activation="relu",
        use_bias=False,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.units = units
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)

    def build(self, input_shape):
        bond_dim = input_shape[0][1]
        atom_dim = input_shape[1][2]

        self.kernel = self.add_weight(
            shape=(bond_dim, atom_dim, self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            trainable=True,
            name="W",
            dtype=tf.float32,
        )

        if self.use_bias:
            self.bias = self.add_weight(
                shape=(bond_dim, 1, self.units),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                trainable=True,
                name="b",
                dtype=tf.float32,
            )

        self.built = True

    def call(self, inputs, training=False):
        adjacency, features = inputs
        # Aggregate information from neighbors
        x = tf.matmul(adjacency, features[:, None, :, :])
        # Apply linear transformation
        x = tf.matmul(x, self.kernel)
        if self.use_bias:
            x += self.bias
        # Reduce bond types dim
        x_reduced = tf.reduce_sum(x, axis=1)
        # Apply non-linear transformation
        return self.activation(x_reduced)

### Actual Disciminator

In [46]:
def GraphDiscriminator(
    gconv_units, dense_units, dropout_rate, adjacency_shape, feature_shape
):

    adjacency = keras.layers.Input(shape=adjacency_shape)
    features = keras.layers.Input(shape=feature_shape)

    # Propagate through one or more graph convolutional layers
    features_transformed = features
    for units in gconv_units:
        features_transformed = RelationalGraphConvLayer(units)(
            [adjacency, features_transformed]
        )

    # Reduce 2-D representation of molecule to 1-D
    x = keras.layers.GlobalAveragePooling1D()(features_transformed)

    # Propagate through one or more densely connected layers
    for units in dense_units:
        x = keras.layers.Dense(units, activation="relu")(x)
        x = keras.layers.Dropout(dropout_rate)(x)

    # For each molecule, output a single scalar value expressing the
    # "realness" of the inputted molecule
    x_out = keras.layers.Dense(1, dtype="float32")(x)

    return keras.Model(inputs=[adjacency, features], outputs=x_out)

In [47]:
NUM_ATOMS = 9  # Maximum number of atoms
ATOM_DIM = 4 + 1  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 64  # Size of the latent space

In [48]:
discriminator = GraphDiscriminator(
    gconv_units=[128, 128, 128, 128],
    dense_units=[512, 512],
    dropout_rate=0.2,
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM),
)

In [49]:
discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 5, 9, 9)]    0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 9, 5)]       0           []                               
                                                                                                  
 relational_graph_conv_layer (R  (None, 9, 128)      3200        ['input_6[0][0]',                
 elationalGraphConvLayer)                                         'input_7[0][0]']                
                                                                                                  
 relational_graph_conv_layer_1   (None, 9, 128)      81920       ['input_6[0][0]',            

## Compile Final Model

In [203]:
generator = GraphGenerator(16, 0.2, 64, (BOND_DIM, NUM_ATOMS, NUM_ATOMS), (NUM_ATOMS, ATOM_DIM), False)

In [206]:
class GraphWGAN(keras.Model):
    def __init__(
        self,
        generator,
        discriminator,
        discriminator_steps=1,
        generator_steps=1,
        gp_weight=10,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.generator = generator
        self.discriminator = discriminator
        self.discriminator_steps = discriminator_steps
        self.generator_steps = generator_steps
        self.gp_weight = gp_weight
        self.latent_dim = 64

    def compile(self, optimizer_generator, optimizer_discriminator, **kwargs):
        super().compile(**kwargs)
        self.optimizer_generator = optimizer_generator
        self.optimizer_discriminator = optimizer_discriminator
        self.metric_generator = keras.metrics.Mean(name="loss_gen")
        self.metric_discriminator = keras.metrics.Mean(name="loss_dis")

    def train_step(self, inputs):

        if isinstance(inputs[0], tuple):
            inputs = inputs[0]

        graph_real = inputs

        self.batch_size = tf.shape(inputs[0])[0]

        # Train the discriminator for one or more steps
        for _ in range(self.discriminator_steps):
            z = tf.random.normal((self.batch_size, self.latent_dim))

            with tf.GradientTape() as tape:
                graph_generated = self.generator(z, training=True)
                loss = self._loss_discriminator(graph_real, graph_generated)

            grads = tape.gradient(loss, self.discriminator.trainable_weights)
            self.optimizer_discriminator.apply_gradients(
                zip(grads, self.discriminator.trainable_weights)
            )
            self.metric_discriminator.update_state(loss)

        # Train the generator for one or more steps
        for _ in range(self.generator_steps):
            z = tf.random.normal((self.batch_size, self.latent_dim))

            with tf.GradientTape() as tape:
                graph_generated = self.generator(z, training=True)
                loss = self._loss_generator(graph_generated)

                grads = tape.gradient(loss, self.generator.trainable_weights)
                self.optimizer_generator.apply_gradients(
                    zip(grads, self.generator.trainable_weights)
                )
                self.metric_generator.update_state(loss)

        return {m.name: m.result() for m in self.metrics}

    def _loss_discriminator(self, graph_real, graph_generated):
        logits_real = self.discriminator(graph_real, training=True)
        logits_generated = self.discriminator(graph_generated, training=True)
        loss = tf.reduce_mean(logits_generated) - tf.reduce_mean(logits_real)
        loss_gp = self._gradient_penalty(graph_real, graph_generated)
        return loss + loss_gp * self.gp_weight

    def _loss_generator(self, graph_generated):
        logits_generated = self.discriminator(graph_generated, training=True)
        return -tf.reduce_mean(logits_generated)

    def _gradient_penalty(self, graph_real, graph_generated):
        # Unpack graphs
        adjacency_real, features_real = graph_real
        adjacency_generated, features_generated = graph_generated

        # Generate interpolated graphs (adjacency_interp and features_interp)
        alpha = tf.random.uniform([self.batch_size])
        alpha = tf.reshape(alpha, (self.batch_size, 1, 1, 1))
        adjacency_interp = (adjacency_real * alpha) + (1 - alpha) * adjacency_generated
        alpha = tf.reshape(alpha, (self.batch_size, 1, 1))
        features_interp = (features_real * alpha) + (1 - alpha) * features_generated

        # Compute the logits of interpolated graphs
        with tf.GradientTape() as tape:
            tape.watch(adjacency_interp)
            tape.watch(features_interp)
            logits = self.discriminator(
                [adjacency_interp, features_interp], training=True
            )

        # Compute the gradients with respect to the interpolated graphs
        grads = tape.gradient(logits, [adjacency_interp, features_interp])
        # Compute the gradient penalty
        grads_adjacency_penalty = (1 - tf.norm(grads[0], axis=1)) ** 2
        grads_features_penalty = (1 - tf.norm(grads[1], axis=2)) ** 2
        return tf.reduce_mean(
            tf.reduce_mean(grads_adjacency_penalty, axis=(-2, -1))
            + tf.reduce_mean(grads_features_penalty, axis=(-1))
        )

In [207]:
QWGAN = GraphWGAN(generator, discriminator)

In [209]:
QWGAN.compile(optimizer_generator=keras.optimizers.Adam(5e-4), optimizer_discriminator=keras.optimizers.Adam(5e-4),)

## Train this MF

In [210]:
QWGAN.fit([adjacency_tensor, feature_tensor], epochs=1, batch_size=16)

ValueError: ignored